In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/My Drive/hafta11')
!pwd

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as k

In [ ]:
# https://www.kaggle.com/deadskull7/fer2013/code
data = pd.read_csv('fer2013.csv')
data.head()

In [ ]:
num_classes = 7   #duygu sayısı
img_width = 48
img_height = 48

emotion_label_to_text = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}

In [ ]:
data.emotion.value_counts()   #hangi duygudan ne kadar var

In [ ]:
# matristen(piksellerden) görüntüye dönüşüm
from matplotlib import pyplot as plt
fig = plt.figure(1, (14, 14))

k = 0
for label in sorted(data.emotion.unique()):
    for j in range(1):
        px = data[data.emotion==label].pixels.iloc[k]
        px = np.array(px.split(' ')).reshape(48, 48).astype('float32')

        k += 1
        ax = plt.subplot(7, 7, k)
        ax.imshow(px , cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(emotion_label_to_text[label])
        plt.tight_layout()

In [ ]:
data.Usage.value_counts()

In [ ]:
# YSA için; giriş, çıkış katmanı belirlendi
# bağımsız değişken: pixels
# bağımlı değişken: emotion
X = data['pixels']
y = data['emotion']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  # test için %20 ayrıldı

In [ ]:
X_train_new = []
for i in X_train:
    X_train_new.append([int(j) for j in i.split()])
X_train_new = np.array(X_train_new)/255.0




X_test_new = []
for i in X_test:
    X_test_new.append([int(j) for j in i.split()])
X_test_new = np.array(X_test_new)/255.0

In [ ]:
# normalleştirme(normalizasyon)
X_train_new = X_train_new.reshape(X_train_new.shape[0], img_width, img_height, 1)

X_test_new = X_test_new.reshape(X_test_new.shape[0], img_width, img_height, 1)

In [ ]:
# yeniden boyutlandırma(flaot32)
X_test_new.shape

X_train_new = X_train_new.astype('float32')
X_test_new = X_test_new.astype('float32')

In [ ]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
# Dense:?
# Flatten:?
# Dropout:?
# MaxPooling:?
# batch: sıralı
# Normalization: karmaşıklık azaltma
from tensorflow.keras.layers import Dense, Conv2D, Flatten,Dropout, MaxPooling2D, Activation, BatchNormalization
from tensorflow.keras.preprocessing import image

In [ ]:
#Katman1
model = Sequential()
# 2 Convolution katmanı: üst üste 2 kesit alındı
model.add(Conv2D(filters=64,kernel_size=(5,5),input_shape=(X_train_new.shape[1:]), padding='same'))
model.add(Conv2D(64, kernel_size=(5,5), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))

In [ ]:
#Katman2  (GİZLİ KATMAN)
model.add(Conv2D(filters=128, kernel_size=(5,5), padding='same'))
model.add(Conv2D(128,kernel_size=(5,5), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))

In [ ]:
#Katman3
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same'))
model.add(Conv2D(256,kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))

In [ ]:
#Katman4
model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))                    #karmaşıklık azaltma

#Katman5: Çıkış katmanı
model.add(Dense(7, activation='softmax'))   #7 nöronlu çıkış üretildi

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# validation başarıyı artırabilir
# Öğrenme -> 3 sa. 1dk. 24 sn. sürdü.
history = model.fit(X_train_new, y_train, epochs=3, batch_size=64, validation_data=(X_test_new,y_test))

In [ ]:
from keras import backend as b
b.set_value(model.optimizer.learning_rate, 0.0001)

pd.DataFrame(history.history).tail()